### Полезные ссылки:

https://developers.google.com/google-ads/api/docs/campaigns/overview

https://developers.google.com/google-ads/api/docs/client-libs/python

https://pypi.org/project/google-ads/

https://github.com/googleads/google-ads-python/#getting-started


https://github.com/googleads/google-ads-python/blob/main/google/ads/googleads/v11/enums/types/advertising_channel_type.py

https://developers.google.com/google-ads/api/docs/client-libs/python/proto-getters

Приложение:

https://console.cloud.google.com/apis/dashboard?hl=ru&project=ace-axon-365313

In [1]:
# pip install google-ads

In [2]:
import pandas as pd
import numpy as np

import argparse
import sys
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException

In [3]:
client = GoogleAdsClient.load_from_storage("google-ads-intapichecking@gmail.com.yaml")

In [4]:
# получение кампаний
# https://developers.google.com/google-ads/api/docs/first-call/get-campaigns#python

def get_campaigns(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
        SELECT
          campaign.id,
          campaign.name
        FROM campaign
        ORDER BY campaign.id"""

    # Issues a search request using streaming.
    stream = ga_service.search_stream(customer_id=customer_id, query=query)
    result = []
    for batch in stream:
#         print(type(batch))
        for row in batch.results:
            result.append({'campaign_id': row.campaign.id,
                             'campaign_name': row.campaign.name})
#             print(
#                 f"Campaign with ID {row.campaign.id} and name "
#                 f'"{row.campaign.name}" was found.'
#             )
    return result

In [5]:
campaigns = get_campaigns(client, customer_id='2200886572')
pd.DataFrame(campaigns)

,campaign_id,campaign_name
0,18616867140,Sales-Search-1


In [6]:
# общий вариант использования отчета
# https://developers.google.com/google-ads/api/docs/reporting/example

def basic_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
        SELECT
          campaign.id,
          campaign.name,
          ad_group.id,
          ad_group.name,
          ad_group_criterion.criterion_id,
          ad_group_criterion.keyword.text,
          ad_group_criterion.keyword.match_type,
          metrics.impressions,
          metrics.clicks,
          metrics.cost_micros
        FROM keyword_view WHERE segments.date DURING LAST_7_DAYS
        AND campaign.advertising_channel_type = 'SEARCH'
        AND ad_group.status = 'ENABLED'
        AND ad_group_criterion.status IN ('ENABLED', 'PAUSED')
        ORDER BY metrics.impressions DESC
        LIMIT 50
        """

    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    result = []
    for batch in stream:
        for row in batch.results:
#             print(row)
            campaign = row.campaign
            ad_group = row.ad_group
            criterion = row.ad_group_criterion
            metrics = row.metrics
            result.append({'Keyword text': criterion.keyword.text,
                           'match type': criterion.keyword.match_type.name,
                           'criterion_ID': criterion.criterion_id,
                           'ad_group': ad_group.name,
                           'group_ID': ad_group.id,
                           'campaign': campaign.name,
                           'campaign_ID': campaign.id,
                           'impressions': metrics.impressions,
                           'clicks': metrics.clicks,
                           'cost_in_micros': metrics.cost_micros})
#             print(
#                 f'Keyword text "{criterion.keyword.text}" with '
#                 f'match type "{criterion.keyword.match_type.name}" '
#                 f"and ID {criterion.criterion_id} in "
#                 f'ad group "{ad_group.name}" '
#                 f'with ID "{ad_group.id}" '
#                 f'in campaign "{campaign.name}" '
#                 f"with ID {campaign.id} "
#                 f"had {metrics.impressions} impression(s), "
#                 f"{metrics.clicks} click(s), and "
#                 f"{metrics.cost_micros} cost (in micros) during "
#                 "the last 7 days."
#             )
                           
    return result

In [7]:
report = basic_report(client, customer_id='2200886572')
pd.DataFrame(report)

,Keyword text,match type,criterion_ID,ad_group,group_ID,campaign,campaign_ID,impressions,clicks,cost_in_micros
0,маркет плейс,BROAD,334480537395,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
1,продажи на маркетплейсах,BROAD,792336893039,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
2,маркетплейс обучение,BROAD,878876771959,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
3,работа с маркетплейсами обучение,BROAD,878876772959,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
4,торговля на маркетплейсах обучение,BROAD,878876773199,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
5,аналитика wildberries,BROAD,890376457331,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
6,обучение работы с wildberries,BROAD,891749857913,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
7,wildberries обучение,BROAD,902566131763,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
8,аналитика продаж на маркетплейсах,BROAD,903990930713,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0
9,менеджер по работе с маркетплейсами обучение,BROAD,903990931873,Группа объявлений 1,145483076147,Sales-Search-1,18616867140,0,0,0


### Кампании

In [8]:
def campaigns_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")
    
    query = """
            SELECT campaign.name,
              campaign_budget.amount_micros,
              campaign.status,
              campaign.optimization_score,
              campaign.advertising_channel_type,
              metrics.clicks,
              metrics.impressions,
              metrics.ctr,
              metrics.average_cpc,
              metrics.cost_micros,
              campaign.bidding_strategy_type
            FROM campaign
            WHERE segments.date DURING LAST_7_DAYS
              AND campaign.status != 'REMOVED'
              """
        # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    
    result = []
    for batch in stream:
        for row in batch.results:
#             print(row)
#             result.append(row)
            campaign = row.campaign
            budget = row.campaign_budget
            metrics = row.metrics
            result.append({'campaign_name': campaign.name,
                          'status': campaign.status.name,
                          'advertising_channel_type': campaign.advertising_channel_type.name,
                          'bidding_strategy_type': campaign.bidding_strategy_type.name,
                          'budget': budget.amount_micros,
                          'clicks': metrics.clicks,
                          'cost_micros': metrics.cost_micros,
                          'impressions': metrics.impressions,
                          'ctr': metrics.ctr,
                           'average_cpc': metrics.average_cpc
                          })
            
    return result

In [9]:
camp_report = campaigns_report(client, customer_id='2200886572')
pd.DataFrame(camp_report)

,campaign_name,status,advertising_channel_type,bidding_strategy_type,budget,clicks,cost_micros,impressions,ctr,average_cpc
0,Sales-Search-1,ENABLED,SEARCH,TARGET_SPEND,500000000,0,0,0,0.0,0.0


### Группы объявлений

In [10]:
def groups_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")
    
    query = """
            SELECT 
              ad_group.id,
              ad_group.name,
              campaign.id,
              campaign.name,
              ad_group.status,
              ad_group.type,
              metrics.clicks,
              metrics.impressions,
              metrics.ctr,
              metrics.average_cpc,
              metrics.cost_micros
            FROM ad_group
            WHERE segments.date DURING LAST_7_DAYS
              AND ad_group.status != 'REMOVED'
              """
    
    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    
    result = []
    for batch in stream:
        for row in batch.results:
#             print(row)
            ad_group = row.ad_group
            campaign = row.campaign
            metrics = row.metrics
            result.append({
                          'ad_group.id': ad_group.id,
                          'ad_group_name': ad_group.name,
                          'campaign_id': campaign.id,
                          'campaign_name': campaign.name,
                          'ad_group_status': ad_group.status.name,
                          'type': ad_group.type_.name,
                          'clicks': metrics.clicks,
                          'cost_micros': metrics.cost_micros,
                          'impressions': metrics.impressions,
                          'ctr': metrics.ctr,
                          'average_cpc': metrics.average_cpc
                          })
    return result

In [11]:
gr_report = groups_report(client, customer_id='2200886572')
pd.DataFrame(gr_report)

,ad_group.id,ad_group_name,campaign_id,campaign_name,ad_group_status,type,clicks,cost_micros,impressions,ctr,average_cpc
0,145483076147,Группа объявлений 1,18616867140,Sales-Search-1,ENABLED,SEARCH_STANDARD,0,0,0,0.0,0.0
1,146496331361,Группа объявлений 2,18616867140,Sales-Search-1,ENABLED,SEARCH_STANDARD,0,0,0,0.0,0.0


### Объявления

In [12]:
def ads_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")
    
    query = """
            SELECT ad_group_ad.ad.expanded_text_ad.headline_part1,
              ad_group_ad.ad.expanded_text_ad.headline_part2,
              ad_group_ad.ad.expanded_text_ad.headline_part3,
              ad_group_ad.ad.final_urls,
              ad_group_ad.ad.expanded_text_ad.description,
              ad_group_ad.ad.expanded_text_ad.description2,
              ad_group_ad.ad.id,
              ad_group_ad.ad.name,
              campaign.id,
              campaign.name,
              ad_group.name,
              ad_group.id,
              ad_group_ad.policy_summary.approval_status,
              ad_group_ad.ad.type,
              metrics.clicks,
              metrics.impressions,
              metrics.ctr,
              metrics.average_cpc,
              metrics.cost_micros,
              metrics.conversions
            FROM ad_group_ad
            WHERE segments.date DURING LAST_7_DAYS
              AND ad_group_ad.status != 'REMOVED'
              """
    
    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    
    result = []
    for batch in stream:
        for row in batch.results:
#             print(row)
            ad_group = row.ad_group
            ad_group_ad = row.ad_group_ad
            campaign = row.campaign
            metrics = row.metrics
            result.append({
                'ad_id': ad_group_ad.ad.id,
                'ad_name': ad_group_ad.ad.name,
                'ad_group_id': ad_group.id,
                'ad_group_name': ad_group.name,
                'ad_approval_status': ad_group_ad.policy_summary.approval_status.name,
                'ad_urls': ad_group_ad.ad.final_urls,
                'ad_type': ad_group_ad.ad.type_.name,
                'campaign': campaign.name,
                'clicks': metrics.clicks,
                'cost_micros': metrics.cost_micros,
                'impressions': metrics.impressions,
                'ctr': metrics.ctr,
                'average_cpc': metrics.average_cpc,
                'conversions': metrics.conversions
                
#                 'text': ad_group_ad.ad.call_ad.headline1
                          })
            
    return result

In [13]:
ad_report = ads_report(client, customer_id='2200886572')
pd.DataFrame(ad_report)

,ad_id,ad_name,ad_group_id,ad_group_name,ad_approval_status,ad_urls,ad_type,campaign,clicks,cost_micros,impressions,ctr,average_cpc,conversions
0,628626126369,,145483076147,Группа объявлений 1,UNKNOWN,[https://ecomru.ru/],RESPONSIVE_SEARCH_AD,Sales-Search-1,0,0,0,0.0,0.0,0.0
1,631227293472,,145483076147,Группа объявлений 1,UNKNOWN,[https://ecomru.ru/],RESPONSIVE_SEARCH_AD,Sales-Search-1,0,0,0,0.0,0.0,0.0


### Расширения

In [14]:
def extensions_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
            SELECT extension_feed_item.sitelink_feed_item.line1,
              extension_feed_item.sitelink_feed_item.line2,
              extension_feed_item.extension_type,
              campaign.name,
              ad_group.name,
              extension_feed_item.status,
              metrics.clicks,
              metrics.impressions,
              metrics.ctr,
              metrics.average_cpc,
              metrics.cost_micros
            FROM extension_feed_item
            WHERE segments.date DURING LAST_7_DAYS
              AND extension_feed_item.status != 'REMOVED'
              """
    
    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    
    result = []
    for batch in stream:
        for row in batch.results:
            print(row)
    
    return result

In [15]:
extensions_report(client, customer_id='2200886572')

[]

### Ключевые слова для поиска

In [16]:
def keywords_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
            SELECT ad_group_criterion.keyword.text,
              campaign.name,
              ad_group.name,
              ad_group_criterion.system_serving_status,
              ad_group_criterion.keyword.match_type,
              ad_group_criterion.approval_status,
              ad_group_criterion.final_urls,
              metrics.clicks,
              metrics.impressions,
              metrics.ctr,
              metrics.average_cpc,
              metrics.cost_micros,
              metrics.all_conversions
            FROM keyword_view
            WHERE segments.date DURING LAST_7_DAYS
              AND ad_group_criterion.status != 'REMOVED'
              """
    
    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    
    result = []
    for batch in stream:
        for row in batch.results:
#             print(row)
            ad_group = row.ad_group
            ad_group_criterion = row.ad_group_criterion
            campaign = row.campaign
            keyword_view = row.keyword_view
            metrics = row.metrics
            result.append({
                'ad_group_name': ad_group.name,
                'system_serving_status': ad_group_criterion.system_serving_status.name,
                'approval_status': ad_group_criterion.approval_status.name,
                'keyword': ad_group_criterion.keyword.text,
                'match_type': ad_group_criterion.keyword.match_type.name,
                'campaign': campaign.name,
                'clicks': metrics.clicks,
                'cost_micros': metrics.cost_micros,
                'impressions': metrics.impressions,
                'ctr': metrics.ctr,
                'average_cpc': metrics.average_cpc,
                'all_conversions': metrics.all_conversions
            })
#     print(query)
    return result


In [17]:
kw_rep = keywords_report(client, customer_id='2200886572')
pd.DataFrame(kw_rep)

,ad_group_name,system_serving_status,approval_status,keyword,match_type,campaign,clicks,cost_micros,impressions,ctr,average_cpc,all_conversions
0,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,маркет плейс,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
1,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,продажи на маркетплейсах,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
2,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,маркетплейс обучение,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
3,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,работа с маркетплейсами обучение,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
4,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,торговля на маркетплейсах обучение,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
5,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,аналитика wildberries,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
6,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,обучение работы с wildberries,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
7,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,wildberries обучение,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
8,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,аналитика продаж на маркетплейсах,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0
9,Группа объявлений 1,ELIGIBLE,UNDER_REVIEW,менеджер по работе с маркетплейсами обучение,BROAD,Sales-Search-1,0,0,0,0.0,0.0,0.0


### Условия поиска

In [18]:
def search_report(client, customer_id):
    ga_service = client.get_service("GoogleAdsService")

    query = """
            SELECT search_term_view.search_term,
              segments.keyword.info.match_type,
              search_term_view.status,
              campaign.name,
              ad_group.name,
              metrics.clicks,
              metrics.impressions,
              metrics.ctr,
              metrics.average_cpc,
              metrics.cost_micros,
              campaign.advertising_channel_type
            FROM search_term_view
            WHERE segments.date DURING LAST_7_DAYS
            """
    
    # Issues a search request using streaming.
    search_request = client.get_type("SearchGoogleAdsStreamRequest")
    search_request.customer_id = customer_id
    search_request.query = query
    stream = ga_service.search_stream(search_request)
    
    result = []
    for batch in stream:
        for row in batch.results:
            print(row)
    
    return result


In [19]:
search_report(client, customer_id='2200886572')

[]